<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice Spark Lab

_Authors: Dave Yerrington (SF)_

---

> *Note: this lab may be best administered as a walkthrough with the instructor.*

**In this lab, we will use Spark to dig into the Bay Area Bike Share data.**

You will need to run this lab on the VM provided. So, as usual, connect to your VM using

    vagrant up
    vagrant ssh

And then, once inside, run:

    spark_local_start.sh

**Important:** If your machine is already running and you've started the Hadoop services with `bigdata_start.sh`, you may want to first run `bigdata_stop.sh` to stop all services and free some memory space.

Once you've started spark in local mode, you should be able to access Jupyter at this address:

http://10.211.55.101:18888

We will work in there.

Our goal is to calculate the average number of trips per hour, using the Caltrain Station as starting point.

**Check that your spark context is available:**

In [ ]:
sc

### 1. Load the Bay Area Bike Share trip data

> **Note:** the data has been pre-loaded on your VM. `201408_babs_open_data/201408_trip_data.csv`

In [ ]:
trips = sc.textFile('file:///home/vagrant/data/201408_babs_open_data/201408_trip_data.csv')

### 2. What kind of object is the data loaded as?

In [1]:
# It is an RDD

### 3. Split csv lines

In spark, we can build complex pipelines that only get executed when we ask to collect them.

In a python pipeline the calculation is immediately executed, but with spark the pipeline definition and execution are separate steps.

In other words, we can define the pipeline with all its steps, and only when we call `collect` will the data flow through it. In order to get familiar with this new workflow, we will start with small steps to build our pipeline.

**Apply a map to trips that splits each line at commas and save that to a an RDD.**

> **Hint:** if you want to check that you're doing things right, you can collect the result and display the first few lines.

In [ ]:
trips = trips.map(lambda line: line.split(","))

### 4. Filter for Caltrain station

In Spark we can also create filters using the `filter` filter method.

**Select station number 70 by filtering on the 5th column.** 

We will do all the following analysis just on this station, which corresponds to the most popular starting point. Save this to a variable called `station_70`.

In [ ]:
station_70 = trips.filter(lambda x: x[4] == '70')

### 5. Trips by day - hour (mapper)

Let's analyse the trips by the hour. We can do this by performing a map reduce job in Spark. First we will need to emit tuples with a count of 1 for each (date, hour) key, and then we will sum the counts by key.

**Emit tuple of ((date, hour), 1), applying a map to `station_70` that extracts the relevant data from each line.**

In [ ]:
# Emit tuple of ((date, hour), 1)
trips_by_day_hour = station_70.map(lambda x: ((x[2].split()[0], x[2].split()[1].split(':')[0]), 1))

### 6. Trips by day - hour (reducer)

Use the `reduceByKey` method to obtain the number of trips per (day, hour).

In [ ]:
trips_by_day_hour = trips_by_day_hour.reduceByKey(lambda a, b: a+b)

### 7. Trips by hour (mapper)

Let's further group the trips by hour. We'll do this with a second Map Reduce job.

First we will discard the day and emit tuples of (hour, count). You can achieve this with a map.

In [ ]:
# Emit tuple of (hour, count)
trips_by_hour = trips_by_day_hour.map(lambda x: (int(x[0][1]), x[1]))

### 8. Trips by hour (reducer)

Now calculate the average number of trips by hour using the `combineByKey` method.

> You can find a suggestion on how to do it [here](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/).

In [ ]:
avg_trips_by_hour = trips_by_hour.combineByKey( (lambda x: (x, 1)), 
 (lambda x, y: (x[0] + y, x[1] + 1)), 
 (lambda x, y: (x[0] + y[0], x[1] + y[1])) 
 )
avg_trips_by_hour = avg_trips_by_hour.mapValues(lambda v : v[0] / v[1]) 

### 9. `collect()` the results.
We can finally collect our result and sort them.

In [ ]:
avg_trips_sorted = sorted(avg_trips_by_hour.collect())

### 10. [Bonus] Using the Spark `sqlContext`

Besides the SparkContext, Spark also exposes a sqlContext that allows us to perform SQL queries on an RDD object.

A SQLContext is also already created for you. Do not create another or unspecified behavior may occur. As you can see below, the sqlContext provided is a HiveContext.

**Run a query using the sqlContext to obtain the average duration of a trip originating from the Caltrain station.**

In [ ]:
sqlContext

In [ ]:
tripsSql = sqlContext.read.format('com.databricks.spark.csv').options(header='true',
                                                                      inferschema='true').load('file:///home/vagrant/data/201408_babs_open_data/201408_trip_data.csv')

In [ ]:
tripsSql.printSchema()

In [ ]:
# Register this DataFrame as a table.
tripsSql.registerTempTable("tripsSql")

In [ ]:
sqlContext.sql("""
SELECT
    hour,
    COUNT(1) AS c,
    ROUND(AVG(duration) / 60) AS avg_duration
FROM (
    SELECT
        CAST(SPLIT(SPLIT(t.startdate, ' ')[1], ':')[0] AS INT) AS hour,
        t.duration AS duration
    FROM "tripsSql" t
    WHERE
        t.startterminal = 70
        AND
        t.duration IS NOT NULL
    ) r
GROUP BY hour
ORDER BY hour ASC;
""")